In [46]:
import os
from os.path import isfile, join, dirname, abspath
import sys
import re
import argparse
import shutil
import subprocess
import tempfile
import pandas as pd

import shapefile
from json import dumps

In [47]:
reader = shapefile.Reader("../data/gis/stations3d.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    # filter by centerline
    
    if 'ENT' in atr['_NAME']:      
        print(atr)
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))
    # write the GeoJSON file

#geojson = open("../data/gis/points_fixes.js", "w")
#geojson.write("var points_fixes = \n")
#geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
#geojson.close()


{'_ID': 1, '_UID': 1, '_NAME': 'ENT_101001', '_SURVEY': '101.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 2, occupation humaine'}
{'_ID': 2, '_UID': 2, '_NAME': 'ENT_102001', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Porche'}
{'_ID': 3, '_UID': 3, '_NAME': 'ENT_102002', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Punta Espolon'}
{'_ID': 4, '_UID': 4, '_NAME': 'ENT_104001', '_SURVEY': '104.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 1, occupation humaine'}
{'_ID': 5, '_UID': 5, '_NAME': 'ENT_104002', '_SURVEY': '104.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Cueva rien'}
{'_ID': 6, '_UID': 6, '_NAME': 'ENT_105001', '_SURVEY': '105.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0,

In [48]:

THERION_STATUS_LIST = [(feature['properties']['_NAME'],feature['properties']['_COMMENT'],feature['geometry']['coordinates'][0],feature['geometry']['coordinates'][1]) for feature in buffer]

In [49]:
THERION_STATUS_LIST

[('ENT_101001', 'Abri 2, occupation humaine', 476162.0, 4461336.0),
 ('ENT_102001', 'Porche', 476260.0, 4449342.0),
 ('ENT_102002', 'Punta Espolon', 474766.0, 4451367.0),
 ('ENT_104001', 'Abri 1, occupation humaine', 475478.0, 4450597.0),
 ('ENT_104002', 'Cueva rien', 475729.0, 4450525.0),
 ('ENT_105001', 'Cueva Rapido1', 482677.0, 4454593.0),
 ('ENT_105002', 'Cueva Rapido 2', 482677.0, 4454593.0),
 ('ENT_106001', 'Cueva Pirates/ Hacienda Salinas', 490220.0, 4452363.0),
 ('ENT_106002', '', 490289.0, 4451286.0),
 ('ENT_106003', '', 490289.0, 4451286.0),
 ('ENT_106004', '', 490541.0, 4453870.0),
 ('ENT_106005', '', 490289.0, 4451286.0),
 ('ENT_106006', '', 490604.0, 4450548.0),
 ('ENT_106008', '', 490080.0, 4452381.0),
 ('ENT_106009', '', 488815.0, 4453011.0),
 ('ENT_106010', '', 488956.0, 4452962.0),
 ('ENT_106011', '', 488956.0, 4452962.0),
 ('ENT_106012', '', 489317.0, 4452812.0),
 ('ENT_106013', '', 489234.0, 4452876.0),
 ('ENT_106014', '', 487636.0, 4452795.0),
 ('ENT_106015', '', 4

In [50]:
## read the cavity synthesis
synthese_cavites = pd.read_csv("../data/BROUILLON_cadastre/UP_MDD_DDA_Temp_synthese_Cavites.csv")

In [51]:
synthese_cavites[['X (UTM 18 Sud)','Y (UTM 18 Sud)','NomCadastre']]

,X (UTM 18 Sud),Y (UTM 18 Sud),NomCadastre
0,473667.0,4416955.0,LeMasqueEtLaPlume
1,469367.0,4421504.0,AbrigoDelViento
2,469510.0,4421914.0,SumideroPiquante
3,470302.0,4421526.0,GouffreAguilucho
4,470298.0,4421336.0,ExsurgencePicaflor
...,...,...,...
444,473093.0,4426314.0,PerteDesSabliers
445,479668.0,4427696.0,ResurgenceLobos
446,479379.0,4428572.0,SystemeCondors
447,479379.0,4428572.0,SystemeCondors


In [52]:
# index

CADASTRE_NUMBER = []
for element in synthese_cavites[['X (UTM 18 Sud)','Y (UTM 18 Sud)','NomCadastre']].iterrows():
    LINE = []
    LINE.append(element[1][2])
    for fixed_point in THERION_STATUS_LIST:
        try:
            if int(fixed_point[2]) == int(element[1][0]):
                if int(fixed_point[3]) == int(element[1][1]):
                    print(fixed_point[0],element[1][2])
                    LINE.append(fixed_point[0])

        except ValueError:
            #print("coordinates are likely NaN")
            pass
    CADASTRE_NUMBER.append(LINE)

ENT_213001 LeMasqueEtLaPlume
ENT_211001 AbrigoDelViento
ENT_211002 SumideroPiquante
ENT_211003 GouffreAguilucho
ENT_211004 ExsurgencePicaflor
ENT_211006 SimaDelCol
ENT_211007 MD-BDGS01
ENT_211008 MD-BDGS02
ENT_211009 MD-BDGS03
ENT_211010 MD-BDGS04
ENT_211011 MD-BDG05
ENT_209001 PleinCintre
ENT_209002 EmergenceBarillet
ENT_209003 GouffreDeLaGachette
ENT_209004 Perte de l'Acharné
ENT_209005 Perte n° 18
ENT_209006 ResurgenceDeLaPlage
ENT_209007 GrotteEvier
ENT_209008 BobLEponge
ENT_209009 CuevaGringa
ENT_208001 Cueva10H10
ENT_205001 AbriCotier
ENT_208002 Résurgence 9 plage de la Baleine
ENT_208003 Grotte20PlageDeLaBaleine
ENT_205002 AbriDuCerfeuil
ENT_206001 CuevaAyayema
ENT_206002 GrotteDuDyke
ENT_205003 CuevaLaBas
ENT_205004 PerteDesDents
ENT_205005 SumideroJuanito
ENT_205006 CuevaArdilla
ENT_205007 PozoEnBosque
ENT_205033 PozoEnBosque
ENT_205008 Pozo 2 en Caliza
ENT_205032 Pozo 2 en Caliza
ENT_205009 Pozo 1 en Caliza
ENT_205034 Pozo 1 en Caliza
ENT_205010 MD-SOU01
ENT_205011 MD-SOU02
E

In [53]:
CADNUM = []
for element in CADASTRE_NUMBER:
    try:
        CADNUM.append(element[1])
    except IndexError:
        print('no cad num attribué')
        CADNUM.append('NAN')
        print(element[0])
        pass

no cad num attribué
PorcheDeLImpatience
no cad num attribué
PequinaSud
no cad num attribué
Perte sans nom
no cad num attribué
Grotte du bout du monde
no cad num attribué
Résurgence 3 plage de la Baleine
no cad num attribué
Résurgence 5 plage de la Baleine
no cad num attribué
ResurgenceDuChampignon
no cad num attribué
CuevaPirates
no cad num attribué
CuevaDeLasLapas
no cad num attribué
GoldenIgue
no cad num attribué
PerteNotoCanada
no cad num attribué
GrotteCacahuete
no cad num attribué
PorcheDeLaCrevette
no cad num attribué
PerteJeanneDArc
no cad num attribué
cueva103
no cad num attribué
Résurgence
no cad num attribué
Résurgence
no cad num attribué
Résurgence
no cad num attribué
GouffreInstantané
no cad num attribué
GouffreDuCanyon
no cad num attribué
LosPiscoleros
no cad num attribué
Traversé du roberto
no cad num attribué
trou noir
no cad num attribué
SumideroArdilla
no cad num attribué
GrotteAudeline
no cad num attribué
Perte1
no cad num attribué
TrouQuiPayePasDeMine
no cad num attr

In [54]:
synthese_cavites['CAD_NUM'] = CADNUM

In [55]:
synthese_cavites

,Unnamed: 0,UP,Nom_complet,X (UTM 18 Sud),Y (UTM 18 Sud),Alt.,Dev. Topo,Dev.estimé,Prof.,Topo.,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,X (UTM 18 Sud).1,Y (UTM 18 Sud).1,Unnamed: 34,Unnamed: 35,NomCadastre,CAD_NUM
0,0,2008,Perte du Masque et de la Plume,473667.0,4416955.0,90,1156.0,NaN,-66,oui,...,NaN,NaN,NaN,NaN,473.667,4416.955,473667.0,4416955.0,LeMasqueEtLaPlume,ENT_213001
1,1,2008,El Abrito del Viento,469367.0,4421504.0,407,430.0,20,-197,oui,...,NaN,NaN,NaN,NaN,469.367,4421.504,469367.0,4421504.0,AbrigoDelViento,ENT_211001
2,2,2008,Sumidero Piquante,469510.0,4421914.0,191,450.0,NaN,-147,oui,...,NaN,NaN,NaN,NaN,469.510,4421.914,469510.0,4421914.0,SumideroPiquante,ENT_211002
3,3,2008,Gouffre d'Aguilucho,470302.0,4421526.0,73,302.0,NaN,-97,oui,...,NaN,NaN,NaN,NaN,470.302,4421.526,470302.0,4421526.0,GouffreAguilucho,ENT_211003
4,4,2008,Exurgence du Picaflor,470298.0,4421336.0,10,40.0,NaN,5,oui,...,NaN,NaN,NaN,NaN,470.298,4421.336,470298.0,4421336.0,ExsurgencePicaflor,ENT_211004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,443,2006,Perte des Sabliers,473093.0,4426314.0,452,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PerteDesSabliers,ENT_209058
445,444,2006,Resurgence des Lobos,479668.0,4427696.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResurgenceLobos,NAN
446,445,2006,Systeme des Condors,479379.0,4428572.0,141,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SystemeCondors,ENT_207001
447,446,2006,Systeme des Condors,479379.0,4428572.0,141,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SystemeCondors,ENT_207001


In [56]:
print(synthese_cavites.columns)

Index(['Unnamed: 0', 'UP', 'Nom_complet', 'X (UTM 18 Sud)', 'Y (UTM 18 Sud)',
       'Alt.', 'Dev. Topo', 'Dev.estimé', 'Prof.', 'Topo.', 'Termine?',
       'Secteur', 'Commentaire', 'Explorateurs', 'ARCHEO', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'X (UTM 18 Sud).1', 'Y (UTM 18 Sud).1', 'Unnamed: 34',
       'Unnamed: 35', 'NomCadastre', 'CAD_NUM'],
      dtype='object')


In [57]:


columns = ['CAD_NUM','NomCadastre','Nom_complet','X (UTM 18 Sud)', 'Y (UTM 18 Sud)','Alt.','Dev. Topo','Termine?', 'Dev.estimé',"Prof.",'UP','Commentaire','Explorateurs']
NOUVELLE_SYNTHESE = synthese_cavites[columns]

In [58]:
NOUVELLE_SYNTHESE.sort_values(by='CAD_NUM').to_csv('../data/BROUILLON_cadastre/UP_MDD_DDA_Temp_synthese_Cavites_NumCad.csv')

In [59]:
MAUVAIS_NOM = []

for dirpath, dirnames, filenames in os.walk('../data'):
    # check along each directory.
    for name in dirnames:
        #print(name)
        # find all cave folders, each has a three digit cadastre sector identifier in path
        if not re.match('\d{3}', name):
            if re.match('.*\d{3}',dirpath) and (name !='outputs'):
                print('yay!', name,dirpath)
                try:
                    print(name)
                    #print(NOUVELLE_SYNTHESE[NOUVELLE_SYNTHESE['NomCadastre']==name])
                except KeyError:
                    pass
                    print(name)
                    print("cette grotte n'est pas dans le xls")
                    MAUVAIS_NOM.append(name)

yay! MD-BDG05 ../data/211
MD-BDG05
yay! GouffreAguilucho ../data/211
GouffreAguilucho
yay! PorcheDeLaMadre2 ../data/211
PorcheDeLaMadre2
yay! AbrigoDelViento ../data/211
AbrigoDelViento
yay! MD-BDGS04 ../data/211
MD-BDGS04
yay! MD-BDGS03 ../data/211
MD-BDGS03
yay! GrottePetiteSoeur ../data/211
GrottePetiteSoeur
yay! PuitsOctopus ../data/211
PuitsOctopus
yay! MD-BDGS02 ../data/211
MD-BDGS02
yay! IgueDeDob ../data/211
IgueDeDob
yay! GouffreDeLArche ../data/211
GouffreDeLArche
yay! GrandAvenTriple ../data/211
GrandAvenTriple
yay! GrotteDuNomade ../data/211
GrotteDuNomade
yay! GrotteGrandeSoeur ../data/211
GrotteGrandeSoeur
yay! GrotteMarion ../data/211
GrotteMarion
yay! AvenDeLaTraverse ../data/211
AvenDeLaTraverse
yay! GrotteDeLaConduite ../data/211
GrotteDeLaConduite
yay! GrotteFilostik ../data/211
GrotteFilostik
yay! GouffreDesMousquetaires ../data/211
GouffreDesMousquetaires
yay! Gouffre36 ../data/211
Gouffre36
yay! PorcheDeLImpatience ../data/211
PorcheDeLImpatience
yay! MeandreDesSa

yay! GB01 ../data/201/C4SansNom4
GB01
yay! Cueva10H10 ../data/208
Cueva10H10
yay! Grotte20PlageDeLaBaleine ../data/208
Grotte20PlageDeLaBaleine


In [60]:
IN_FOLDER = []
for NAME in NOUVELLE_SYNTHESE['NomCadastre']:
    
    COND = False
    for dirpath, dirnames, filenames in os.walk('../data'):
    # check along each directory.
        for name in dirnames:
            if NAME in name:
                print(NAME,name,dirpath)
                COND = True
    IN_FOLDER.append((NAME,COND))

LeMasqueEtLaPlume LeMasqueEtLaPlume ../data/213
AbrigoDelViento AbrigoDelViento ../data/211
SumideroPiquante SumideroPiquante ../data/211
GouffreAguilucho GouffreAguilucho ../data/211
ExsurgencePicaflor ExsurgencePicaflor ../data/211
PorcheDeLImpatience PorcheDeLImpatience ../data/211
SimaDelCol SimaDelCol ../data/211
MD-BDGS01 MD-BDGS01 ../data/211
MD-BDGS02 MD-BDGS02 ../data/211
MD-BDGS03 MD-BDGS03 ../data/211
MD-BDGS04 MD-BDGS04 ../data/211
MD-BDG05 MD-BDG05 ../data/211
PleinCintre GrotteDuPleinCintre ../data/209
EmergenceBarillet EmergenceBarillet ../data/209
GouffreDeLaGachette GouffreDeLaGachette ../data/209
ResurgenceDeLaPlage ResurgenceDeLaPlage ../data/209
GrotteEvier GrotteEvier ../data/209
BobLEponge BobLEponge ../data/209
CuevaGringa CuevaGringa ../data/209
Cueva10H10 Cueva10H10 ../data/208
AbriCotier AbriCotier ../data/205
Grotte20PlageDeLaBaleine Grotte20PlageDeLaBaleine ../data/208
AbriDuCerfeuil AbriDuCerfeuil ../data/205
CuevaAyayema CuevaAyayema ../data/206
GrotteDuDy

Cueva6 C4Cueva6 ../data/201
Perte1Lobos Perte1Lobos ../data/211
GrotteAudeline GrotteAudeline ../data/205
PerteDesLobos PerteDesLobos ../data/211
PerteDesLobos PerteDesLobos ../data/ovni
ElCuloDeLaMadre ElCuloDeLaMadre ../data/211
LaGrotte LaGrotte ../data/211
IgueDeDob IgueDeDob ../data/211
GrottePetiteSoeur GrottePetiteSoeur ../data/211
PuitsOctopus PuitsOctopus ../data/211
GouffreDuKit GouffreDuKit ../data/209
GouffreCumpleAnos GouffreCumpleAnos ../data/211
GouffreCumpleAnos GouffreCumpleAnos ../data/209
LaMarmotte LaMarmotte ../data/209
AvenKinka4 AvenKinka4 ../data/209
Perte3 Perte3 ../data/201
AvenKinka3 AvenKinka3 ../data/209
Aven GrandAvenTriple ../data/211
Aven AvenDeLaTraverse ../data/211
Aven ResurgenceDeLAvenir ../data/304
Aven AvenKinka4 ../data/209
Aven AvenKinka3 ../data/209
Aven AvenKinka2 ../data/209
Aven AvenKinka1 ../data/209
Aven GrandAvenPerte ../data/209
Aven GrandAvenTriple ../data/211
Aven AvenDeLaTraverse ../data/211
Aven ResurgenceDeLAvenir ../data/304
Aven Av

EnDesespoirDeCause EnDesespoirDeCause ../data/109
P10 P10 ../data/109
PetiteGrotte PetiteGrotte ../data/109
GouffreHalal GouffreHalal ../data/109
Gouffralail Gouffralail ../data/109
GouffreMaPremiere GouffreMaPremiere ../data/109
GrandeTraversee GrandeTraversee ../data/109
CuevaDeLaMerienda CuevaDeLaMerienda ../data/109
Gauges Gauges ../data/109
Pissotiere Pissotiere ../data/109
TroisEntreesPlusUne TroisEntreesPlusUne ../data/109
QuatreEntreesPlusUne QuatreEntreesPlusUne ../data/109
PuntaBlanca PuntaBlanca ../data/107
TrouDeLaChattiere TrouDeLaChattiere ../data/109
ResurgenceHongo ResurgenceHongo ../data/107
PN01 PN01 ../data/107
PN02 PN02 ../data/107
GouffreJackpot GouffreJackpot ../data/107
PerteCirque4 PerteCirque4 ../data/107
PetiteChinoise PetiteChinoise ../data/109
CuevaCascada CuevaCascada ../data/107
ParaisoDeLasDuchas ParaisoDeLasDuchas ../data/109
MoulinReveBleu MoulinReveBleu ../data/401
CuevaEsperanza CuevaEsperanza ../data/401
Traversee GrandeTraversee ../data/109
Resurgen

In [61]:
A_REVOIR = NOUVELLE_SYNTHESE[pd.DataFrame(IN_FOLDER, columns = ['NAME','COND'])['COND']==False]

In [62]:
A_REVOIR.to_csv('../data/BROUILLON_cadastre/Cavites_Non_Attribuees.csv')

In [63]:
A_REVOIR[A_REVOIR['CAD_NUM']=="NAN"].dropna(subset = ['X (UTM 18 Sud)','Y (UTM 18 Sud)']).to_csv('../data/BROUILLON_cadastre/NonAttribue.csv')

In [89]:
with open('../data/209/209.th', 'r') as f:
    data = f.readlines()
    for line in data:
        if '##' in line:
            root = line.split('##')[0]
            
            suffix = line.split('##')[1]
            print(root+ '\n \tstation\t'+ root.split('fix')[1].split('\t')[0]+'\t"{}"\n'.format(suffix.split('\n')[0].strip(' ')))
    f.close()

    fix ENT_209001	473616	4425884	215.0	
 	station	 ENT_209001	""

    fix ENT_209002	472922	4425932	436.0	
 	station	 ENT_209002	""

    fix ENT_209003	473027	4425922	428.0	
 	station	 ENT_209003	"Gouffre de la Gachette"

    fix ENT_209004	473027	4425450	334.0	
 	station	 ENT_209004	"Perte de l'Acharné"

    fix ENT_209005	473067	4425846	422.0	
 	station	 ENT_209005	"Perte n?? 18"

    fix ENT_209006	471328	4422992	3.0	
 	station	 ENT_209006	"Résurgence de la plage"

    fix ENT_209008	473372	4427993	200.0	
 	station	 ENT_209008	"Bob l'Eponge"

    fix ENT_209009	472980	4428693	154.0	
 	station	 ENT_209009	"Cueva Gringa (Los Gringos)"

    fix ENT_209010	473125	4428203	250.0	
 	station	 ENT_209010	"Doline Calafate"

    fix ENT_209011	473050	4425902	445.0	
 	station	 ENT_209011	""

    fix ENT_209012	473026	4425860	450.0	
 	station	 ENT_209012	"Aven"

    fix ENT_209013	473542	4426174	357.0	
 	station	 ENT_209013	"Aven"

    fix ENT_209014	470653	4425395	505.0	
 	station	 ENT_209014	